In [ ]:
# Stock Market Sentiment Analysis with Sector Tagging & Impact Prediction

import feedparser
from transformers import pipeline
import torch
import datetime
import time
import re
import csv

# -------------------------
# SETUP
# -------------------------
RSS_FEED = "https://news.google.com/rss/search?q=stock+market+india&hl=en-IN&gl=IN&ceid=IN:en"
CHECK_INTERVAL = 300  # seconds (5 minutes)

# Load pre-trained sentiment model
sentiment_analyzer = pipeline("sentiment-analysis")

# Sector tagging keywords
KEYWORDS = {
    "Banking": ["bank", "PSU", "sbi", "pnb", "hdfc", "icici"],
    "IT": ["TCS", "Infosys", "Wipro", "Tech Mahindra"],
    "Auto": ["Tata Motors", "Maruti", "EV", "Mahindra"],
    "Finance": ["Ministry of Finance", "RBI", "monetary policy", "budget", "repo rate"],
    "Energy": ["Oil", "ONGC", "Reliance Energy", "Adani Green"],
    "Metals": ["Steel", "Tata Steel", "JSW", "Hindalco"]
}

seen_titles = set()

# -------------------------
# HELPER FUNCTIONS
# -------------------------
def classify_sector(text):
    text = text.lower()
    for sector, keywords in KEYWORDS.items():
        for kw in keywords:
            if kw.lower() in text:
                return sector
    return "General"

def estimate_impact(score, sentiment):
    if sentiment == "POSITIVE":
        return "Potential Upside (10-20%)"
    elif sentiment == "NEGATIVE":
        return "Potential Downside (10-20%)"
    else:
        return "Neutral / Low Impact"

# -------------------------
# MAIN MONITOR LOOP
# -------------------------
print("\nStarting Sector-wise Real-Time Stock News Sentiment Monitor...")

# Create CSV file to store output
with open("news_sentiment_log.csv", "w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Time", "Headline", "Sector", "Sentiment", "Score", "Impact", "Link"])

    while True:
        feed = feedparser.parse(RSS_FEED)
        print(f"\nChecked at: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

        for entry in feed.entries:
            title = entry.title
            link = entry.link
            published = entry.published

            if title in seen_titles:
                continue

            seen_titles.add(title)
            clean_title = re.sub(r'[^\w\s]', '', title)
            sentiment = sentiment_analyzer(clean_title)[0]

            label = sentiment['label']
            score = sentiment['score']
            sector = classify_sector(title)
            impact = estimate_impact(score, label)

            print("\nNEW NEWS ALERT")
            print(f"Headline: {title}")
            print(f"Published: {published}")
            print(f"Link: {link}")
            print(f"Sentiment: {label} ({score:.2f})")
            print(f"Sector: {sector}")
            print(f"Predicted Market Impact: {impact}")

            # Save to CSV
            writer.writerow([
                published, title, sector, label, round(score, 2), impact, link
            ])
            file.flush()

        time.sleep(CHECK_INTERVAL)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Device set to use cpu



Starting Sector-wise Real-Time Stock News Sentiment Monitor...

Checked at: 2025-07-30 07:30:39

NEW NEWS ALERT
Headline: Why is the India-US trade deal delayed? What does it mean for the Indian stock market? Explained - Mint
Published: Wed, 30 Jul 2025 04:35:13 GMT
Link: https://news.google.com/rss/articles/CBMi9wFBVV95cUxNeExTMlA1ZXFDTXlFUXhGVWpLV2I1TnBIZFh0Ykdxdk4wZFk4UzBYcDIxRFVyNmpkQzc5bzltZHA2eXh0bGxaazA1d28xSWJBSWloenlPaXFkYkJLc25zd0UwckhoVllpanpCUEhHNF9qb0pxUjYxSDlMd3NXMkFyUXNZU2xZTkc3RnpEUEJlVlRCTVNEcXB5VHVkNmIwMUc0d1RRcm5yVHdpMDVrSldfLU1hOExOT1JKZEk3Mm10NF81aFI4dU84OXkxamV4bDFNM2pMR0dfM0tmeHpXeVFxTk1fODNmZWhabHJxMW52WWMwUnBzVENj0gH8AUFVX3lxTFB4VnFPSUxvTTc0M3F0dGlTaTctODAyOTdWcFpGS295RE1KdFRHQXJ1WFB1WUJnbTRTSC05MEdYVi1ReDlCdlVNV3phUl9fOHdkTjBrN2NEdlVfQ19WUzkyLTZrb29DblZSNm45Y0FyS2VJaFczeXZMZlUwWWhuN0pjaUs5VXp5VUY4TDhMS0thVzNfMXhnLU5aNkpkVlJCU0c1bGJtVnNvbThmaW5MMXkwMVBvdWptWWhjSVA1SXphbVE0ZkZzdU5kLUVoSktWUW43VGtRZmxVZEcxLW1yXy1jeFZfQ3dDY25sa1pXd054UFpJNFdqVjhwbUFNdQ?oc=5
Senti